In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

# Load in Data

We'll load the Pride and Prejudice text file, an 1813 romantic novel of manners written by Jane Austen, and convert it into integers for our network to use.

In [2]:
# open text file and read in data as `text`
with open('data/pride-and-prejudice.txt', 'r') as f:
    text = f.read()
    text = ''.join([i for i in text if not i.isdigit()])

Let's check out the first 100 characters, make sure everything is peachy.

In [3]:
text[:100]

'It is a truth universally acknowledged, that a single man in\npossession of a good fortune, must be i'

# Tokenization
We'll create a dictionary out of all the characters that we have in the sentences and map them to an integer. This will allow us to convert our input characters to their respective integers (char2int) and vice versa (int2char).

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

First few encoded characters

In [5]:
encoded[:100]

array([12, 42, 49, 46, 44, 49,  5, 49, 42, 62, 28, 42, 68, 49, 28, 27, 46,
       10, 50, 62, 44,  5, 43, 43, 24, 49,  5, 25,  0, 27,  7, 53, 43, 50,
       21, 67, 50, 21, 34, 49, 42, 68,  5, 42, 49,  5, 49, 44, 46, 27, 67,
       43, 50, 49,  6,  5, 27, 49, 46, 27, 57, 58,  7, 44, 44, 50, 44, 44,
       46,  7, 27, 49,  7, 20, 49,  5, 49, 67,  7,  7, 21, 49, 20,  7, 62,
       42, 28, 27, 50, 34, 49,  6, 28, 44, 42, 49, 51, 50, 49, 46])

# Pre-processing the data
LSTM expects an input that is one-hot encoded meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that.

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


# Creating batches

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

# Test Your Implementation
Now I'll make some data sets and we can check out what's going on as we batch data. 
Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[12 42 49 46 44 49  5 49 42 62]
 [68 50  6 49 27 50 48 42 49 42]
 [44 28 25 68 49  5 57 25 68  5]
 [50 42 42 50 62 49 42  7 49 68]
 [34 49 42 68  5 42 34 49 62 50]
 [50 49 68 50  5 62 21 49  6 28]
 [43 50 49 42  7 49 20 46 27 21]
 [ 5 27 25 50 49  7 20 49 58 43]]

y
 [[42 49 46 44 49  5 49 42 62 28]
 [50  6 49 27 50 48 42 49 42  7]
 [28 25 68 49  5 57 25 68  5 62]
 [42 42 50 62 49 42  7 49 68 50]
 [49 42 68  5 42 34 49 62 50 67]
 [49 68 50  5 62 21 49  6 28 25]
 [50 49 42  7 49 20 46 27 21 49]
 [27 25 50 49  7 20 49 58 43 50]]


Before we start building the model, let's use a built-in feature in PyTorch to check the device we're running on (CPU or GPU). This implementation will not require GPU as the training is really simple. However, as you progress on to large datasets and models with millions of trainable parameters, using the GPU will be very important to speed up your training.

In [10]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `drop_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

### Creating a training method
The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [12]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    valid_loss_min = np.Inf 
    
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                
                if (np.mean(val_losses) < valid_loss_min) and (loss.item() < valid_loss_min) :
                    print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, np.mean(val_losses)))
                    # change the name, for saving multiple files
                    model_name = 'rnn_20_epoch.net'

                    checkpoint = {'n_hidden': net.n_hidden,
                                  'n_layers': net.n_layers,
                                  'state_dict': net.state_dict(),
                                  'tokens': net.chars}

                    with open(model_name, 'wb') as f:
                        torch.save(checkpoint, f)
                        valid_loss_min = np.mean(val_losses)
                

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [13]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(72, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=72, bias=True)
)


In [14]:
batch_size = 128
seq_length = 100
n_epochs = 100 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/100... Step: 10... Loss: 3.2629... Val Loss: 3.2323
Validation loss decreased (inf --> 3.232285). Saving model...
Epoch: 1/100... Step: 20... Loss: 3.1705... Val Loss: 3.1567
Validation loss decreased (3.232285 --> 3.156722). Saving model...
Epoch: 1/100... Step: 30... Loss: 3.1326... Val Loss: 3.1412
Validation loss decreased (3.156722 --> 3.141207). Saving model...
Epoch: 1/100... Step: 40... Loss: 3.1448... Val Loss: 3.1369
Epoch: 2/100... Step: 50... Loss: 3.1314... Val Loss: 3.1343
Validation loss decreased (3.141207 --> 3.134296). Saving model...
Epoch: 2/100... Step: 60... Loss: 3.1239... Val Loss: 3.1316
Validation loss decreased (3.134296 --> 3.131593). Saving model...
Epoch: 2/100... Step: 70... Loss: 3.1190... Val Loss: 3.1263
Validation loss decreased (3.131593 --> 3.126325). Saving model...
Epoch: 2/100... Step: 80... Loss: 3.1081... Val Loss: 3.1163
Validation loss decreased (3.126325 --> 3.116331). Saving model...
Epoch: 2/100... Step: 90... Loss: 3.0905... Val 

Epoch: 21/100... Step: 980... Loss: 1.4068... Val Loss: 1.3227
Epoch: 21/100... Step: 990... Loss: 1.3730... Val Loss: 1.3191
Epoch: 21/100... Step: 1000... Loss: 1.3919... Val Loss: 1.3100
Epoch: 22/100... Step: 1010... Loss: 1.3656... Val Loss: 1.3114
Epoch: 22/100... Step: 1020... Loss: 1.3678... Val Loss: 1.3073
Epoch: 22/100... Step: 1030... Loss: 1.3853... Val Loss: 1.3034
Epoch: 22/100... Step: 1040... Loss: 1.3605... Val Loss: 1.2994
Epoch: 22/100... Step: 1050... Loss: 1.3776... Val Loss: 1.2948
Epoch: 23/100... Step: 1060... Loss: 1.3382... Val Loss: 1.2949
Validation loss decreased (1.344307 --> 1.294901). Saving model...
Epoch: 23/100... Step: 1070... Loss: 1.3290... Val Loss: 1.2955
Epoch: 23/100... Step: 1080... Loss: 1.3125... Val Loss: 1.2901
Epoch: 23/100... Step: 1090... Loss: 1.3343... Val Loss: 1.2842
Epoch: 23/100... Step: 1100... Loss: 1.3231... Val Loss: 1.2837
Epoch: 24/100... Step: 1110... Loss: 1.3530... Val Loss: 1.2811
Epoch: 24/100... Step: 1120... Loss: 1.

Epoch: 43/100... Step: 2060... Loss: 1.1143... Val Loss: 1.1753
Validation loss decreased (1.175391 --> 1.175265). Saving model...
Epoch: 44/100... Step: 2070... Loss: 1.1337... Val Loss: 1.1723
Validation loss decreased (1.175265 --> 1.172323). Saving model...
Epoch: 44/100... Step: 2080... Loss: 1.1252... Val Loss: 1.1724
Epoch: 44/100... Step: 2090... Loss: 1.1528... Val Loss: 1.1705
Validation loss decreased (1.172323 --> 1.170482). Saving model...
Epoch: 44/100... Step: 2100... Loss: 1.1292... Val Loss: 1.1725
Epoch: 44/100... Step: 2110... Loss: 1.1313... Val Loss: 1.1693
Validation loss decreased (1.170482 --> 1.169298). Saving model...
Epoch: 45/100... Step: 2120... Loss: 1.1324... Val Loss: 1.1692
Validation loss decreased (1.169298 --> 1.169189). Saving model...
Epoch: 45/100... Step: 2130... Loss: 1.1111... Val Loss: 1.1721
Epoch: 45/100... Step: 2140... Loss: 1.1430... Val Loss: 1.1668
Validation loss decreased (1.169189 --> 1.166779). Saving model...
Epoch: 45/100... Step:

Epoch: 67/100... Step: 3180... Loss: 0.9954... Val Loss: 1.1897
Epoch: 67/100... Step: 3190... Loss: 0.9989... Val Loss: 1.1756
Epoch: 67/100... Step: 3200... Loss: 0.9847... Val Loss: 1.1725
Epoch: 67/100... Step: 3210... Loss: 0.9835... Val Loss: 1.1842
Epoch: 68/100... Step: 3220... Loss: 0.9715... Val Loss: 1.1796
Epoch: 68/100... Step: 3230... Loss: 0.9561... Val Loss: 1.1835
Epoch: 68/100... Step: 3240... Loss: 0.9422... Val Loss: 1.1791
Epoch: 68/100... Step: 3250... Loss: 0.9728... Val Loss: 1.1831
Epoch: 68/100... Step: 3260... Loss: 0.9659... Val Loss: 1.1837
Epoch: 69/100... Step: 3270... Loss: 0.9655... Val Loss: 1.1802
Epoch: 69/100... Step: 3280... Loss: 0.9657... Val Loss: 1.1861
Epoch: 69/100... Step: 3290... Loss: 0.9823... Val Loss: 1.1830
Epoch: 69/100... Step: 3300... Loss: 0.9710... Val Loss: 1.1931
Epoch: 69/100... Step: 3310... Loss: 0.9771... Val Loss: 1.1818
Epoch: 70/100... Step: 3320... Loss: 0.9766... Val Loss: 1.1857
Epoch: 70/100... Step: 3330... Loss: 0.9

Epoch: 93/100... Step: 4460... Loss: 0.8572... Val Loss: 1.2598
Epoch: 94/100... Step: 4470... Loss: 0.8551... Val Loss: 1.2457
Epoch: 94/100... Step: 4480... Loss: 0.8468... Val Loss: 1.2689
Epoch: 94/100... Step: 4490... Loss: 0.8662... Val Loss: 1.2532
Epoch: 94/100... Step: 4500... Loss: 0.8657... Val Loss: 1.2498
Epoch: 94/100... Step: 4510... Loss: 0.8475... Val Loss: 1.2634
Epoch: 95/100... Step: 4520... Loss: 0.8682... Val Loss: 1.2518
Epoch: 95/100... Step: 4530... Loss: 0.8407... Val Loss: 1.2768
Epoch: 95/100... Step: 4540... Loss: 0.8796... Val Loss: 1.2610
Epoch: 95/100... Step: 4550... Loss: 0.8759... Val Loss: 1.2613
Epoch: 95/100... Step: 4560... Loss: 0.8932... Val Loss: 1.2696
Epoch: 96/100... Step: 4570... Loss: 0.8566... Val Loss: 1.2517
Epoch: 96/100... Step: 4580... Loss: 0.8496... Val Loss: 1.2761
Epoch: 96/100... Step: 4590... Loss: 0.8785... Val Loss: 1.2690
Epoch: 96/100... Step: 4600... Loss: 0.8608... Val Loss: 1.2732
Epoch: 97/100... Step: 4610... Loss: 0.8

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [15]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [16]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [17]:
print(sample(net, 1000, prime='The', top_k=5))

Thers.

They had supposed him twented into her manner about
their corpers, and had not seen him at Notherâ€”â€”â€”shire;
and as they were now satisfied with them.

Mrs. Bennet had not been so few misler by his friend. Mr. Gardinerâ€™s
repeated painf, answered their favourable care. She was
soon an indelicacy sight in the room. She was soon
able to remain. They were something of her own."

"I have no resentment for anything. He must have been satisfied."

"I have not at her spress in his parish and the ladies
she must have the sense of it.
Though I am abreat of any other party."

Elizabeth was a sort to be influlted by the house, and her
mind could not have had ever been treatment; and her put and
consequence which had been soon as soon as he was away, the
first of all that he had received an advice so far as she could
tris; he must spinith intelligence. His first possibility
of all they were attended to him. Her services there was no
ling in the pain of the subject. They could not be f

## Loading a checkpoint

In [18]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [23]:
# Sample using a loaded model
print(sample(loaded, 1000, top_k=5, prime="you must know"))

you must know that
he had not being in in the proposation. The
expression of her feirness was allowed to see him, and was
an answer, and though such a second thoughts were always
superior in her friend. His approaching wo and see such
dependence as she was shance and serious two young ladies and
the plannes at this circumstances, and as they always affected how at acquaintatce with her
father to hear and so many in the solitity of a personâ€™s course of
attentions, so soon as they were than soon father, and with so mome and the
case talked, and town when they had never saw, and she continued
to be so delighted for the concern of his sisterâ€™s
placting. Mrs. Collins could not his suspect. She had not been always
paused to houre, and so sure of her dear sid, she went along and
the samo story at this county; and which her sister was a
solicitity for their father and doing that shopted his perpossitious is to
be dancing to say what had said there soon an appearing
for her faces, and she c